### 유튜브 동영상의 여러가지 품질

In [ ]:
from urllib.request import Request, urlopen
import json

header = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.88 Safari/537.36",
    "Referer": "https://youtube.com"
}

video_id = "3-mHctyi3_Y"
watch_url = "https://www.youtube.com/watch?v={}".format(video_id)

watch_html = urlopen(watch_url).read().decode("utf-8")
start_str = "ytInitialPlayerResponse = "
start_index = watch_html.find(start_str)
end_index = watch_html.find("};", start_index + 1) if start_index >= 0 else 0

if start_index < end_index:
    h = watch_html[start_index + len(start_str):end_index + 1]
    _json = json.loads(h)
    aformats = _json['streamingData']['adaptiveFormats']
    for f in aformats:
        print("itag:{}, type:{}, quality:{}, url:{}".format(f["itag"], f["mimeType"], f["quality"], f["url"][:50]))

위 코드는 ```_json['streamingData']``` 에서 ```['adaptiveFormats']``` 항목의 요소들을 출력하는 샘플 코드 입니다. ```url``` 요소의 값은 너무 길어서 가독성을 위해 50자로 제한해서 출력하게 했습니다. 위의 코드를 실행해보면 우리에게 필요로 하는 실제 동영상의 주소는 ['streamingData']['adaptiveFormats'] 항목에 타입별 리스트 형태로 있다는 사실을 알았습니다. 또한 ```type```을 보면 해당 주소에 대한 데이터가 video 데이터인지 audio 데이터인지 아니면 video/audio 데이터인지도 알수 있습니다. video 만 존재하는 데이터인 경우에는 video + audio 를 모두 다운로드 하여 다른 유틸리티를 사용하여 비디오파일과 오디오 파일을 병합해야 하는 번거로움이 있습니다.

In [ ]:
from urllib.request import Request, urlopen
import json

def find_list(listdata, key, value):
    for item in listdata:
        if item.get(key) == value:
            return item
    return None

header = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.88 Safari/537.36",
    "Referer": "https://youtube.com"
}

video_id = "3-mHctyi3_Y"
watch_url = "https://www.youtube.com/watch?v={}".format(video_id)

watch_html = urlopen(watch_url).read().decode("utf-8")
start_str = "ytInitialPlayerResponse = "
start_index = watch_html.find(start_str)
end_index = watch_html.find("};", start_index + 1) if start_index >= 0 else 0

if start_index < end_index:
    h = watch_html[start_index + len(start_str):end_index + 1]
    _json = json.loads(h)
    aformats = _json['streamingData']['adaptiveFormats']

    # find_list 함수 사용
    print(find_list(aformats, "itag", 137))

    # 파이썬의 next() 함수 사용
    print(next((item for item in aformats if item["itag"] == 137), None))

### 특정 요소 추출

<pre style="background-color:#eeeeee;margin:0px;padding:10px;margin-top:15px;">
def find_list(listdata, key, value):
    for item in listdata:
        if item.get(key) == value:
            return item
    return None

data = find_list(aformats, "itag", 137)

혹은

data = find_list(aformats, "quality", "hd1080")
</pre>

우리는 위의 코드처럼 ```aformats``` 변수 안에 있는 여러 항목중에 쉽게 원하는 키:값에 해당하는 데이터를 구하는 ```find_list``` 함수를 작성할 수 있습니다. 

<pre style="background-color:#eeeeee;margin:0px;padding:10px;margin-top:15px;">
data = next((item for item in aformats if item["itag"] == 137), None)
</pre>

```find_list``` 함수를 작성하지 않고 파이썬의 ```next()``` 함수를 사용하여 더 간결하게 작성할 수도 있습니다. 위의 코드는 방금 작성한 ```find_list()``` 함수의 기능을 대신할 수 있습니다.

### 동영상 다운로드 샘플

In [ ]:
from urllib.request import Request, urlopen
import json

header = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.88 Safari/537.36",
    "Referer": "https://youtube.com"
}

video_id = "3-mHctyi3_Y"
watch_url = "https://www.youtube.com/watch?v={}".format(video_id)

watch_html = urlopen(watch_url).read().decode("utf-8")
start_str = "ytInitialPlayerResponse = "
start_index = watch_html.find(start_str)
end_index = watch_html.find("};", start_index + 1) if start_index >= 0 else 0

if start_index < end_index:
    h = watch_html[start_index + len(start_str):end_index + 1]
    _json = json.loads(h)
    aformats = _json['streamingData']['adaptiveFormats']
    data = next((item for item in aformats if item["itag"] == 137), None)
    if data is not None:
        sample_url = data["url"]
        quality = data["quality"]
        print("동영상 주소 >>>>>>>>", sample_url, quality)

        request = Request(sample_url, headers=header)
        r = urlopen(request)
        filesize = int(r.headers["Content-Length"])
        filename = "{}.mp4".format(video_id)
        print("다운로드 사이즈 >>>>>>>", filesize)

        download_size = 0
        block_size = 8129
        with open(filename, "wb") as f:
            while True:
                buffer = r.read(block_size)
                if not buffer:
                    break
                download_size += len(buffer)
                f.write(buffer)
                status = "{:10d} [{:03.2f}%%]".format(download_size, download_size * 100.0 / filesize)
                status += chr(8) * (len(status) + 1)
                print(status)
        print(">>>> 다운로드 완료 <<<<")

위의 코드는 지금까지 내용을 바탕으로 유튜브에서 ```itag``` 가 137 인 1080 화질의 mp4 포맷 파일을 하나 다운로드 하는 샘플입니다. 

<pre style="background-color:#eeeeee;margin:0px;padding:10px;margin-top:15px;">
data = next((item for item in aformats if item["itag"] == 137), None)
</pre>

우리는 ```aformats```변수에 저장되어있는 여러 요소들중 itag 가 137 인 1080 화질의 mp4 비디오 영상을 선택해서 ```data``` 변수에 저장하였습니다.

<pre style="background-color:#eeeeee;margin:0px;padding:10px;margin-top:15px;">
sample_url = data["url"]
</pre>

저장된 항목에서 우리에게 필요한 ```url``` 키 값을 구해 변수에 저장합니다. 이 ```url``` 이 실제 동영상 주소 입니다.

<pre style="background-color:#eeeeee;margin:0px;padding:10px;margin-top:15px;">
request = Request(sample_url, headers=header)
r = urlopen(request)
filesize = int(r.headers["Content-Length"])
filename = "{}.mp4".format(video_id)
</pre>

동영상 주소로 접속을 하고 먼저 헤더에서 ```Content-Length``` 값을 구해 현재 서버로부터 전송받을 데이터의 용량을 구해옵니다. 파일명은 단순히 유튜브 동영상의 인덱스 값으로 정했습니다.

<pre style="background-color:#eeeeee;margin:0px;padding:10px;margin-top:15px;">
download_size = 0
block_size = 8129
</pre>
```download_size``` 변수는 다운로드 된 총 양을 저장하기 위해 사용되고 ```block_size``` 변수는 서버로부터 한번에 전송받을 데이터의 양을 설정하기 위해 사용됩니다. 적당한 값을 설정하면 되는데 이 값이 무조건 다운로드 양을 보장하지는 않습니다.

<pre style="background-color:#eeeeee;margin:0px;padding:10px;margin-top:15px;">
with open(filename, "wb") as f:
    while True:
        buffer = r.read(block_size)
        if not buffer:
            break
        download_size += len(buffer)
        f.write(buffer)
        status = "{:10d} [{:03.2f}%%]".format(download_size, download_size * 100.0 / filesize)
        status += chr(8) * (len(status) + 1)
        print(status)
</pre>

파일을 ```"wb"``` 바이너리 쓰기 모드로 열고 서버로부터 더이상 받을 데이터가 없을때까지 루프를 돌면서 ```buffer = r.read(block_size)``` 동영상 데이터를 전송 받아 ```f.write(buffer)``` 파일에 기록합니다. ```status``` 변수에는 현재 다운로드의 상태를 안내하기 상태 정보가 저장되어있습니다. 그러나 이 작업은 비디오 파일만 저장되었기 때문에 추가로 오디오 파일도 저장을 해야 합니다.

In [ ]:
from urllib.request import Request, urlopen
import json

def download_url(data, filename):
    if data is None:
        print("Data is None")
        return None
    sample_url = data["url"]
    quality = data["quality"]
    print("다운로드 주소 >>>>>>>>", sample_url, quality)

    request = Request(sample_url, headers=header)
    r = urlopen(request)
    filesize = int(r.headers["Content-Length"])
    save_filename = "{}.mp4".format(filename)
    print("다운로드 사이즈 >>>>>>>", filesize)

    download_size = 0
    block_size = 8129
    with open(save_filename, "wb") as f:
        while True:
            buffer = r.read(block_size)
            if not buffer:
                break
            download_size += len(buffer)
            f.write(buffer)
            status = "{:10d} [{:03.2f}%%]".format(download_size, download_size * 100.0 / filesize)
            status += chr(8) * (len(status) + 1)
            print(status)
    print(">>>> {} 다운로드 완료 <<<<".format(filename))

header = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/79.0.3945.88 Safari/537.36",
    "Referer": "https://youtube.com"
}

video_id = "3-mHctyi3_Y"
watch_url = "https://www.youtube.com/watch?v={}".format(video_id)

watch_html = urlopen(watch_url).read().decode("utf-8")
start_str = "ytInitialPlayerResponse = "
start_index = watch_html.find(start_str)
end_index = watch_html.find("};", start_index + 1) if start_index >= 0 else 0

if start_index < end_index:
    h = watch_html[start_index + len(start_str):end_index + 1]
    _json = json.loads(h)
    aformats = _json['streamingData']['adaptiveFormats']
    video = next((item for item in aformats if item["itag"] == 137), None)
    audio = next((item for item in aformats if item["itag"] == 140), None)

    download_url(video, "{}_video".format(video_id))
    download_url(audio, "{}_audio".format(video_id))

위 코드는 기존의 코드에서 다운로드 하는 부분을 따로 ```def download_url()``` 이라는 함수로 분리하고 비디오만 있는 1080 의 mp4 형식인 itag 137 과 오디오만 있는 itag 140 을 다운로드 하는 내용으로 수정한 코드 입니다. 위의 코드를 실행하면 비디오는 비디오ID_video.mp4 로 저장되고 오디오는 비디오ID_audio.mp4 로 저장이 됩니다.